# LSTM

In [ ]:
import yfinance as yf
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

# Fetch stock data
data = yf.download("GOOG", start="2012-01-01", end="2024-01-01")  # Change the ticker if needed
close_prices = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_prices)

# Prepare training data (using past 30 days to predict next day)
X_train, y_train = [], []
for i in range(30, len(scaled_data)):
    X_train.append(scaled_data[i-30:i, 0])
    y_train.append(scaled_data[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1)
y_train = torch.tensor(y_train, dtype=torch.float32)

# Define LSTM model
class LSTMStockPredictor(nn.Module):
    def __init__(self):
        super(LSTMStockPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=2, batch_first=True)
        self.fc = nn.Linear(50, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  # Take last output
        return x

# Initialize model
model = LSTMStockPredictor()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 10
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs.squeeze(), y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

# Save the trained model
torch.save(model.state_dict(), "lstm_stock_model.pth")

# Save the scaler for later use
import joblib
joblib.dump(scaler, "scaler.pkl")

print("Model saved as lstm_stock_model.pth")


[*********************100%***********************]  1 of 1 completed


Epoch 1/10, Loss: 0.203770
Epoch 2/10, Loss: 0.188315
Epoch 3/10, Loss: 0.173950
Epoch 4/10, Loss: 0.160530
Epoch 5/10, Loss: 0.147930
Epoch 6/10, Loss: 0.136068
Epoch 7/10, Loss: 0.124906
Epoch 8/10, Loss: 0.114437
Epoch 9/10, Loss: 0.104697
Epoch 10/10, Loss: 0.095769
Model saved as lstm_stock_model.pth


# XGBOOST


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Prepare the data
X, y = [], []
for i in range(60, len(scaled_data)):
    X.append(scaled_data[i-60:i, 0])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=500, learning_rate=0.01)
xgb_model.fit(X_train, y_train)

# Forecast next 7 days
future_X = X[-1:]
xgb_predictions = []
for _ in range(7):
    pred = xgb_model.predict(future_X)
    xgb_predictions.append(pred[0])
    future_X = np.append(future_X[:, 1:], pred).reshape(1, 60)

# Rescale predictions
xgb_predictions = scaler.inverse_transform(np.array(xgb_predictions).reshape(-1, 1)).flatten()

# Display predictions
print("XGBoost Predictions for Next 7 Days:", xgb_predictions)


# LIGHTGBM

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datetime import datetime

# Step 1: Fetch the latest stock data
today = datetime.now().strftime('%Y-%m-%d')
data = yf.download('GOOG', start='2012-01-01', end=today)

# Step 2: Prepare the data
close_prices = data['Close']
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_prices.values.reshape(-1, 1))

# Step 3: Prepare training and testing datasets
X, y = [], []
for i in range(60, len(scaled_data)):
    X.append(scaled_data[i-60:i, 0])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train LightGBM model
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_test = lgb.Dataset(X_test, label=y_test, reference=lgb_train)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'early_stopping_round': 50  # Early stopping rounds in the params dictionary
}

lgb_model = lgb.train(params, lgb_train, valid_sets=[lgb_test], num_boost_round=1000)

# Step 5: Forecast the next 7 days
X_last_60 = scaled_data[-60:].reshape(1, -1)
lgb_predictions = []
for _ in range(7):
    pred = lgb_model.predict(X_last_60)
    lgb_predictions.append(pred[0])
    X_last_60 = np.append(X_last_60[0][1:], pred).reshape(1, -1)

# Rescale predictions back to original prices
lgb_predictions = scaler.inverse_transform(np.array(lgb_predictions).reshape(-1, 1)).flatten()

# Step 6: Plot the results
future_dates = pd.date_range(datetime.now().date(), periods=7, freq='B')
plt.figure(figsize=(12, 6))
plt.plot(close_prices[-200:], label="Actual")
plt.plot(future_dates, lgb_predictions, label="LGBM Predictions", linestyle='--')
plt.title('GOOGLE Stock Price Prediction for Next 7 Days (LightGBM)')
plt.xlabel('Date')
plt.ylabel('Stock Price (USD)')
plt.legend()
plt.show()

# Display predictions
print(f"LGBM Predictions for Next 7 Days: {lgb_predictions}")


# GRU

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf

# Step 1: Fetch Google stock price data up to today
data = yf.download('GOOG', start='2012-01-01', end=pd.Timestamp.today().strftime('%Y-%m-%d'))
data = data[['Close']]  # Use only the closing price
data.index = pd.to_datetime(data.index)

# Step 2: Scale data for GRU model
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Prepare training data
seq_length = 60  # Use the last 60 days to predict the next day
X, y = [], []
for i in range(seq_length, len(scaled_data)):
    X.append(scaled_data[i-seq_length:i])
    y.append(scaled_data[i])
X, y = np.array(X), np.array(y)

# Step 3: Build and train the GRU model
model = Sequential([
    GRU(50, return_sequences=True, input_shape=(seq_length, 1)),
    GRU(50),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=20, batch_size=32, verbose=1)

# Step 4: Forecast the next 7 days
last_sequence = scaled_data[-seq_length:]  # Last 60 days as input
last_sequence = last_sequence.reshape(1, seq_length, 1)  # Reshape for model input

future_prices = []
for _ in range(7):
    next_price = model.predict(last_sequence)[0, 0]  # Predict next price
    future_prices.append(next_price)
    next_sequence = np.append(last_sequence[0, 1:], [[next_price]], axis=0)
    last_sequence = next_sequence.reshape(1, seq_length, 1)

# Convert scaled predictions back to original prices
future_prices_actual = scaler.inverse_transform(np.array(future_prices).reshape(-1, 1))

# Step 5: Create future dates
future_dates = pd.date_range(start=pd.Timestamp.today(), periods=8)[1:]  # Next 7 days
forecast_df = pd.DataFrame({'Date': future_dates, 'Forecasted Price': future_prices_actual.flatten()})

# Step 6: Visualization
plt.figure(figsize=(12, 6))
plt.plot(data.index[-100:], data['Close'].values[-100:], label="Historical Prices")
plt.plot(forecast_df['Date'], forecast_df['Forecasted Price'], label="Forecasted Prices", marker='o')
plt.title("Google Stock Price Forecast for the Next 7 Days")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.show()

# Print forecasted prices
print("Forecasted Prices for the Next 7 Days:")
print(forecast_df)


# SARIMAX

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Fit SARIMAX model
model = SARIMAX(close_prices, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
results = model.fit()

# Forecast the next 7 days
forecast = results.get_forecast(steps=7)
forecast_df = forecast.summary_frame()

# Display predictions
print(forecast_df[['mean', 'mean_ci_lower', 'mean_ci_upper']])

Discord Bot

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2syEU0lluGcpWCczw8FxLEcr69i_5Qc7TjGYU5tcjguPDUbNH")

In [ ]:
!pip install discord.py nest-asyncio yfinance numpy pandas tensorflow scikit-learn xgboost
import discord
import nest_asyncio
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from discord.ext import commands

# Required for Google Colab to run async functions
nest_asyncio.apply()


In [ ]:
import discord
import asyncio
import nest_asyncio
import yfinance as yf
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam
from discord.ext import commands

# Allow async in Google Colab
nest_asyncio.apply()

# Discord Bot Setup
TOKEN = "MTMzOTQxMjk4Nzc1NjI4NjA1NQ.GMeU50.qenW_KcCND_XeT0dwr1wAJ-n6MUDyDh8x4GtaU"
intents = discord.Intents.default()
intents.messages = True
intents.message_content = True
bot = commands.Bot(command_prefix="!", intents=intents)

@bot.event
async def on_ready():
    print(f"✅ Logged in as {bot.user}")

@bot.command()
async def hello(ctx):
    await ctx.send("Hello!")

# Function to fetch stock data
def fetch_stock_data(ticker):
    stock = yf.Ticker(ticker)
    df = stock.history(period="1y")  # Get past year data
    return df if not df.empty else None

# Preprocess data
def preprocess_data(data):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[['Close']])
    return scaled_data, scaler

# Prepare sequences (Fix: Predict only next day's price)
def prepare_sequences(data, seq_length=30):
    X, y = [], []
    for i in range(len(data) - seq_length - 1):  # Predict only the next day
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])  # Single value target
    return np.array(X), np.array(y)

# Build LSTM Model
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, activation='relu', return_sequences=True, input_shape=input_shape),
        LSTM(50, activation='relu'),
        Dense(1)  # Predict 1 day
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Build GRU Model
def build_gru_model(input_shape):
    model = Sequential([
        GRU(50, activation='relu', return_sequences=True, input_shape=input_shape),
        GRU(50, activation='relu'),
        Dense(1)  # Predict 1 day
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Train LSTM/GRU model
def train_model(model, X_train, y_train):
    model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)
    return model

# Train XGBoost (Fix: Ensure correct target shape)
def train_xgboost(X_train, y_train):
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)
    model.fit(X_train.reshape(X_train.shape[0], -1), y_train.ravel())  # Ensure correct shape
    return model

# Train LightGBM (Fix: Ensure correct target shape)
def train_lightgbm(X_train, y_train):
    model = lgb.LGBMRegressor(n_estimators=100)
    model.fit(X_train.reshape(X_train.shape[0], -1), y_train.ravel())  # Ensure correct shape
    return model

# Train SARIMAX
def train_sarimax(data):
    model = SARIMAX(data, order=(1,1,1), seasonal_order=(1,1,1,12))
    result = model.fit(disp=False)
    return result

# Predict stock prices
def predict_stock(ticker):
    print(f"📊 Predicting for {ticker}...")

    # Fetch data
    data = fetch_stock_data(ticker)
    if data is None:
        return f"⚠️ No data found for {ticker}"

    # Preprocess
    data_scaled, scaler = preprocess_data(data)
    X, y = prepare_sequences(data_scaled)
    X_train, y_train = X[:-1], y[:-1]  # Use all but the last for training
    X_test = X[-1].reshape(1, *X.shape[1:])  # Use the last sequence for prediction

    # Train models
    lstm_model = train_model(build_lstm_model((30,1)), X_train, y_train)
    gru_model = train_model(build_gru_model((30,1)), X_train, y_train)
    xgb_model = train_xgboost(X_train, y_train)
    lgb_model = train_lightgbm(X_train, y_train)
    sarimax_model = train_sarimax(data['Close'])

    # Predictions (Fix: Reshape correctly)
    predictions = {
        'LSTM': scaler.inverse_transform(lstm_model.predict(X_test).reshape(-1,1))[0],
        'GRU': scaler.inverse_transform(gru_model.predict(X_test).reshape(-1,1))[0],
        'XGBoost': scaler.inverse_transform(xgb_model.predict(X_test.reshape(1,-1)).reshape(-1,1))[0],
        'LightGBM': scaler.inverse_transform(lgb_model.predict(X_test.reshape(1,-1)).reshape(-1,1))[0],
        'SARIMAX': scaler.inverse_transform(sarimax_model.forecast(steps=1).values.reshape(-1,1))[0],
    }

    # Format results
    result = f"📈 **Predictions for {ticker}** for the next 7 days\n"
    result += f"Current Price: **{data['Close'].iloc[-1]:.2f}**\n"
    for model, pred in predictions.items():
        result += f"**{model}**: {pred[0]:.2f}\n"

    return result

# Discord command
@bot.command()
async def predict(ctx, ticker: str):
    try:
        print(f"📌 Command received: !predict {ticker}")
        result = predict_stock(ticker.upper())
        await ctx.send(result)
    except Exception as e:
        print(f"❌ Error: {e}")
        await ctx.send(f"Error: {e}")

# Run the bot
async def run_bot():
    async with bot:
        await bot.start(TOKEN)

loop = asyncio.get_event_loop()
loop.run_until_complete(run_bot())


2025-02-13 05:29:39 INFO     discord.client logging in using static token
2025-02-13 05:29:39 INFO     discord.client logging in using static token
2025-02-13 05:29:39 INFO     discord.client logging in using static token
2025-02-13 05:29:39 INFO     discord.client logging in using static token
2025-02-13 05:29:39 INFO     discord.client logging in using static token
2025-02-13 05:29:39 INFO     discord.client logging in using static token
2025-02-13 05:29:39 INFO     discord.client logging in using static token
2025-02-13 05:29:39 INFO     discord.client logging in using static token
2025-02-13 05:29:39 INFO     discord.client logging in using static token
2025-02-13 05:29:39 INFO     discord.client logging in using static token
2025-02-13 05:29:39 INFO     discord.client logging in using static token
2025-02-13 05:29:39 INFO     discord.client logging in using static token
2025-02-13 05:29:39 INFO     discord.client logging in using static token
2025-02-13 05:29:39 INFO     discord.c

✅ Logged in as Compare AI#0380
📌 Command received: !predict TSLA
📊 Predicting for TSLA...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000425 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2220
[LightGBM] [Info] Number of data points in the train set: 219, number of used features: 30
[LightGBM] [Info] Start training from score 0.348537
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


KeyboardInterrupt: 